In [15]:
import tensorflow as tf
import os
import cv2
import numpy as np
import random

In [1]:
# Set dataset path
# Load list of class names
with open("../Datasets/UCF101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/classInd.txt") as f:
    class_names = [line.strip().split()[1] for line in f.readlines()]
len(class_names)

101

In [17]:
# # def vid_to_np_arr(filename,filepath,resized_cls_dir):
# def vid_to_np_arr(filepath):
#     # Load the video file
#     # resized_file_path = "../Datasets/UCF101 Resized/"
#     video = cv2.VideoCapture(filepath)
#     frames = []
#     while video.isOpened():
#         ret, frame = video.read()
#         if not ret:
#             break
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         frame = cv2.resize(frame, (64,64))
#         # Save the resized image
#         # file_path = resized_cls_dir + '/' + filename
#         # cv2.imwrite(file_path, frame)
#         frames.append(np.array(frame))
#     np_arry = np.array(frames)
#     video.release()
#     return np_arry

In [18]:
# # Load training data
# train_data = []
# train_labels = []
# resized_ds_dir = "../Datasets/UCF101_Resized/UCF101/UCF-101/"
# for class_id, class_name in enumerate(class_names):
#     class_dir = os.path.join('../Datasets/UCF101/UCF101/UCF-101/',class_name)
#     # os.makedirs(resized_ds_dir+class_name)
#     # resized_cls_dir = resized_ds_dir + class_name
#     for filename in os.listdir(class_dir)[:20]:
#         filepath = os.path.join(class_dir, filename)
#         video = np.load(filepath)
#         # video = vid_to_np_arr(filepath)
#         # video = vid_to_np_arr(filename,filepath,resized_cls_dir)
#         train_data.append(video)
#         train_labels.append(class_id)

In [19]:
# # Load testing data
# test_data = []
# test_labels = []
# with open("../Datasets/UCF101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/testlist01.txt") as f:
#     for line in f.readlines()[:20]:
#         filename = line.strip().split()[0]
#         class_name = filename.split("/")[0]
#         class_id = class_names.index(class_name)
#         filepath = os.path.join(filename)
#         video = np.load(filepath)
#         # video = vid_to_np_arr(filepath)
#         test_labels.append(class_id)

In [20]:
subset_paths={}
base_path = '../Datasets/UCF101/UCF-101/'
train_files = os.listdir('../Datasets/UCF101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/')
# print(train_files)
for file in train_files[:-1]:
    if 'train' in file:
        with open("../Datasets/UCF101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/"+file, "r") as f:
            # print(file)
            subset_paths['train'] = [base_path+line.strip().split()[0] for line in f.readlines()]
            # print([line.split(' ') for line in f.readlines()])
            # subset_paths['train_labels'] = [line.strip().split()[1] for line in f.readlines()] 
    if 'testlist01' in file or 'testlist02' in file:
        with open("../Datasets/UCF101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/"+file, "r") as f:
            # print(file)
            subset_paths['val'] = [base_path+line.strip() for line in f.readlines()]
    if 'testlist03' in file:
        with open("../Datasets/UCF101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/"+file, "r") as f:
            # print(file)
            subset_paths['test'] = [base_path+line.strip() for line in f.readlines()]

random.shuffle(subset_paths['train'])
# subset_paths['train']=subset_paths['train'][:50]
random.shuffle(subset_paths['test'])
# subset_paths['test']=subset_paths['test'][:50]
random.shuffle(subset_paths['val'])
# subset_paths['val']=subset_paths['val'][:50]
# print(subset_paths['test'][:4])
# # print(subset_paths['train_labels'][:4])
# print(subset_paths['val'][:4])
# print(subset_paths['train'][:4])

In [21]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame


In [22]:
def frames_from_video_file(video_path, n_frames, output_size = (128,128), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result


In [23]:
class FrameGenerator:
  def __init__(self, path, n_frames, training = False,key='train'):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names =  class_names
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))
    self.key = key
  def get_files_and_class_names(self):
    return subset_paths[self.key], self.class_names

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label


In [24]:
# Create the training set
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['train'], 10, training=True,key='train'),
                                          output_signature = output_signature)
# Create the validation set
val_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['val'], 10, key='val'),
                                        output_signature = output_signature)

test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['test'], 10, key='test'),
                                        output_signature = output_signature)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
train_ds = train_ds.batch(2)
val_ds = val_ds.batch(2)
test_ds = test_ds.batch(2)

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

test_frames, test_labels = next(iter(test_ds))
print(f'Shape of test set of frames: {test_frames.shape}')
print(f'Shape of test labels: {test_labels.shape}')


2023-04-26 05:35:13.991200: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 67 of 1000
2023-04-26 05:35:18.804213: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


Shape of training set of frames: (2, 10, 128, 128, 3)
Shape of training labels: (2,)


2023-04-26 05:35:28.954762: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 76 of 1000
2023-04-26 05:35:32.328389: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


Shape of validation set of frames: (2, 10, 128, 128, 3)
Shape of validation labels: (2,)


2023-04-26 05:35:42.436313: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 70 of 1000


Shape of test set of frames: (2, 10, 128, 128, 3)
Shape of test labels: (2,)


2023-04-26 05:35:46.703923: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


In [25]:

# # Convert data to tensors and normalize pixel values to range [0, 1]
# train_data = tf.convert_to_tensor(train_data, dtype=tf.float32) / 255.
# train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
# test_data = tf.convert_to_tensor(test_data, dtype=tf.float32) / 255.
# test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

# # Create dataset objects
# train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(len(train_data)).batch(32)
# test_ds = tf.data.Dataset.from_tensor_slices((test_data, test_labels)).batch(32)

In [31]:

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', input_shape=(10, 128, 128, 3)),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Conv3D(61, (3, 3, 3), activation='relu'),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    # tf.keras.layers.Conv3D(61, (3, 3, 3), activation='relu'),
    # tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 8, 126, 126, 64)   5248      
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 4, 63, 63, 64)    0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 2, 61, 61, 61)     105469    
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 1, 30, 30, 61)    0         
 3D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 54900)             0         
                                                                 
 dense_4 (Dense)             (None, 512)              

In [29]:

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
model.fit(train_ds, epochs=50,validation_data=val_ds)


Epoch 1/50
51/51 [==============================] - 5s 60ms/step - loss: 4.3875 - accuracy: 0.0396 - val_loss: 4.6156 - val_accuracy: 0.0198
Epoch 2/50
51/51 [==============================] - 3s 57ms/step - loss: 4.2569 - accuracy: 0.0693 - val_loss: 4.7166 - val_accuracy: 0.0000e+00
Epoch 3/50
51/51 [==============================] - 3s 57ms/step - loss: 3.8091 - accuracy: 0.1683 - val_loss: 4.9672 - val_accuracy: 0.0099
Epoch 4/50
51/51 [==============================] - 3s 57ms/step - loss: 2.4771 - accuracy: 0.4653 - val_loss: 6.9977 - val_accuracy: 0.0099
Epoch 5/50
51/51 [==============================] - 3s 57ms/step - loss: 1.4846 - accuracy: 0.6634 - val_loss: 8.9558 - val_accuracy: 0.0297
Epoch 6/50
51/51 [==============================] - 3s 57ms/step - loss: 0.9460 - accuracy: 0.7723 - val_loss: 8.0387 - val_accuracy: 0.0099
Epoch 7/50
51/51 [==============================] - 3s 57ms/step - loss: 0.6518 - accuracy: 0.8317 - val_loss: 13.2826 - val_accuracy: 0.0099
Epoch 8/

In [30]:

# Evaluate the model on the test set
model.evaluate(test_ds)


51/51 [==============================] - 1s 19ms/step - loss: 27.6904 - accuracy: 0.0099


[27.690393447875977, 0.009900989942252636]